<a href="https://colab.research.google.com/github/skj092/Lab/blob/main/DirectoryPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%bash 

cd /content
cp kaggle.json /root/.kaggle 
chmod 600 /root/.kaggle/kaggle.json 
kaggle datasets download -d solesensei/solesensei_bdd100k

100%|██████████| 7.61G/7.61G [01:26<00:00, 94.2MB/s]


In [4]:
!pwd 

/content


In [5]:
!unzip /content/solesensei_bdd100k.zip 

Streaming output truncated to the last 5000 lines.
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3cf340ac-4e445717_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d06c567-00000000_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d0d454e-f0132c99_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d12f470-45096fb4_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d187d59-48023a43_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d1a6305-3980e3bc_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d2c1ef5-fdc7bea5_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d3117f0-9365aac0_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d317eb1-4a9f2f5c_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d31c23f-bb0e2813_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg/labels/train/3d31d9a7-2fa8d37a_train_id.png  
  inflating: bdd100k_seg/bdd100k/seg

In [6]:
!mkdir images
!mkdir labels

# BDD json to COCO Json

In [9]:
import os
import json
import argparse
from tqdm import tqdm



def bdd2coco_detection(id_dict, labeled_images, fn):

    images = list()
    annotations = list()

    counter = 0
    for i in tqdm(labeled_images):
        counter += 1
        image = dict()
        image['file_name'] = i['name']
        image['height'] = 720
        image['width'] = 1280

        image['id'] = counter

        empty_image = True

        for label in i['labels']:
            annotation = dict()
            category=label['category']
            if (category == "traffic light"):
                color = label['attributes']['trafficLightColor']
                category = "tl_" + color
            if category in id_dict.keys():
                empty_image = False
                annotation["iscrowd"] = 0
                annotation["image_id"] = image['id']
                x1 = label['box2d']['x1']
                y1 = label['box2d']['y1']
                x2 = label['box2d']['x2']
                y2 = label['box2d']['y2']
                annotation['bbox'] = [x1, y1, x2-x1, y2-y1]
                annotation['area'] = float((x2 - x1) * (y2 - y1))
                annotation['category_id'] = id_dict[category]
                annotation['ignore'] = 0
                annotation['id'] = label['id']
                annotation['segmentation'] = [[x1, y1, x1, y2, x2, y2, x2, y1]]
                annotations.append(annotation)

        if empty_image:
            continue

        images.append(image)

    attr_dict["images"] = images
    attr_dict["annotations"] = annotations
    attr_dict["type"] = "instances"

    print('saving...')
    json_string = json.dumps(attr_dict)
    with open(fn, "w") as file:
        file.write(json_string)


if __name__ == '__main__':

    label_dir="/content/bdd100k_labels_release/bdd100k/labels"
    save_path="labels/"

    attr_dict = dict()
    attr_dict["categories"] = [
        {"supercategory": "none", "id": 1, "name": "person"},
        {"supercategory": "none", "id": 2, "name": "rider"},
        {"supercategory": "none", "id": 3, "name": "car"},
        {"supercategory": "none", "id": 4, "name": "bus"},
        {"supercategory": "none", "id": 5, "name": "truck"},
        {"supercategory": "none", "id": 6, "name": "bike"},
        {"supercategory": "none", "id": 7, "name": "motor"},
        {"supercategory": "none", "id": 8, "name": "tl_green"},
        {"supercategory": "none", "id": 9, "name": "tl_red"},
        {"supercategory": "none", "id": 10, "name": "tl_yellow"},
        {"supercategory": "none", "id": 11, "name": "tl_none"},
        {"supercategory": "none", "id": 12, "name": "traffic sign"},
        {"supercategory": "none", "id": 13, "name": "train"}
    ]

    attr_id_dict = {i['name']: i['id'] for i in attr_dict['categories']}

    # # create BDD training set detections in COCO format
    # print('Loading training set...')
    # with open(os.path.join(label_dir,
    #                        'bdd100k_labels_images_train.json')) as f:
    #     train_labels = json.load(f)
    # print('Converting training set...')

    # out_fn = os.path.join(save_path,
    #                       'bdd100k_labels_images_det_coco_train.json')
    # bdd2coco_detection(attr_id_dict, train_labels, out_fn)

    print('Loading validation set...')
    # create BDD validation set detections in COCO format
    with open(os.path.join(label_dir,
                           'bdd100k_labels_images_val.json')) as f:
        val_labels = json.load(f)
    print('Converting validation set...')

    out_fn = os.path.join(save_path,
                          'bdd100k_labels_images_det_coco_val.json')
    bdd2coco_detection(attr_id_dict, val_labels, out_fn)

Loading validation set...
Converting validation set...


100%|██████████| 10000/10000 [00:01<00:00, 6437.29it/s]


saving...


In [10]:
!mkdir preprocessed

In [11]:
!mkdir preprocessed/labels

In [14]:
!mv /content/drive/MyDrive/Projects/all_data /content/drive/MyDrive/Projects/BDD/

In [16]:
import os
from xml.etree.ElementTree import dump
import json
import pprint

import argparse

from format import COCO,YOLO

def main(config):

    if config["datasets"] == "COCO":
        coco = COCO()
        yolo = YOLO(os.path.abspath(config["cls_list"]))

        flag, data = coco.parse(config["label"])

        if flag == True:
            flag, data = yolo.generate(data)

            if flag == True:
                flag, data = yolo.save(data, config["output_path"], config["img_path"],
                                        config["img_type"], config["manipast_path"])

                if flag == False:
                    print("Saving Result : {}, msg : {}".format(flag, data))

            else:
                print("YOLO Generating Result : {}, msg : {}".format(flag, data))

        else:
            print("COCO Parsing Result : {}, msg : {}".format(flag, data))

    else:
        print("Unkwon Datasets")

if __name__ == '__main__':

#     config ={
#         "datasets": "COCO",
#         "img_path": "bdd100k_images/bdd100k/images/100k/train",
#         "label": "labels/bdd100k_labels_images_det_coco_train.json",
#         "img_type": ".jpg",
#         "manipast_path": "./",
#         "output_path": "labels/trains/",
#         "cls_list": "data/bdd100k.names",
#     }
    config ={
        "datasets": "COCO",
        "img_path": "/content/drive/MyDrive/Projects/BDD/all_data",
        "label": "/content/labels/bdd100k_labels_images_det_coco_val.json",
        "img_type": ".jpg",
        "manipast_path": "./",
        "output_path": "preprocessed/",
        "cls_list": "bdd100k.names",
    }

    main(config)


COCO Parsing:  |████████████████████████████████████████| 100.0% (185526/185526)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (10000/10000)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (10000/10000)  Complete



# Filter our the CAR label

In [17]:
# This script will delete all the labels from the yolo labels except car 

import os
import glob
import shutil
from tqdm import tqdm 

# Path to the folder containing the yolo labels
path = '/content/preprocessed'

# Path to the folder where you want to save the modified labels
save_path = '/content/preprocessed'

# label code for car
car = 2

for filename in tqdm(os.listdir(path)):
    # continue if the file is classes.txt
    if filename == 'classes.txt':
        continue
    if filename.endswith(".txt"):
        with open(os.path.join(path, filename), 'r') as f:
            lines = f.readlines()
            f.close()
        with open(os.path.join(save_path, filename), 'w') as f:
            for line in lines:
                if int(line.split(' ')[0]) == car:
                    f.write(line)
            f.close()

100%|██████████| 10001/10001 [00:01<00:00, 5525.66it/s]


# Replacing 2 with 0 in the all labels 

In [19]:
# open the text file and update the first entry 

import os
import glob
import shutil
from tqdm import tqdm

# Path to the folder containing the yolo labels
path = '/content/preprocessed'

all_text_files = glob.glob(path + '/*.txt')
print('len(all_text_files):', len(all_text_files))

for filename in tqdm(all_text_files):
    if filename.endswith(".txt"):
        with open(filename, 'r') as f:
            lines = f.readlines()
            f.close()
        with open(filename, 'w') as f:
            for line in lines:
                line = line[:1].replace('2', '0') + line[1:]
                f.write(line)
            f.close()

len(all_text_files): 10000


100%|██████████| 10000/10000 [00:01<00:00, 6331.20it/s]


# Split the data into training and validatio 

In [20]:
from glob import glob
image_path_list = glob('/content/drive/MyDrive/Projects/BDD/all_data/*.jpg')
label_path_list = glob('/content/preprocessed/*.txt')

print('number of images: ', len(image_path_list))
print('number of labels: ', len(label_path_list))


number of images:  10000
number of labels:  10000


In [22]:
!cp -r /content/preprocessed /content/drive/MyDrive/Projects/BDD

In [23]:
cd /content/drive/MyDrive/Projects/BDD

/content/drive/MyDrive/Projects/BDD


In [25]:
!ls

all_data  preprocessed	valid-lbls.zip	val-imgs.zip


In [26]:
# This script will split the data into training and validation sets

import os
import random
import shutil
from glob import glob
from tqdm import tqdm

# creating folder for training and validation
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('valid'):
    os.makedirs('valid')

# creading folder for images and labels
if not os.path.exists('train/images'):
    os.makedirs('train/images')
if not os.path.exists('train/labels'):
    os.makedirs('train/labels')

# creading folder for images and labels
if not os.path.exists('valid/images'):
    os.makedirs('valid/images')
if not os.path.exists('valid/labels'):
    os.makedirs('valid/labels')



# Creating partitions of the data after shuffeling
# Get the list of all images
image_path_list = glob('/content/drive/MyDrive/Projects/BDD/all_data/*.jpg')
label_path_list = glob('/content/drive/MyDrive/Projects/BDD/preprocessed/*.txt')

print('number of images: ', len(image_path_list))
print('number of labels: ', len(label_path_list))

# Shuffle the images
random.shuffle(image_path_list)
random.shuffle(label_path_list)

# Calculate the split size
train_size = int(0.9 * len(image_path_list))
valid_size = int(0.1 * len(image_path_list))

# Copy-pasting images
train_image_list = image_path_list[:train_size]
valid_image_list = image_path_list[train_size:train_size+valid_size]

train_label_list = label_path_list[:train_size]
valid_label_list = label_path_list[train_size:train_size+valid_size]

# Train
for image_path in tqdm(train_image_list):
    shutil.copy(image_path, 'train/images')

for label_path in tqdm(train_label_list):
    shutil.copy(label_path, 'train/labels')

# Validation
for image_path in tqdm(valid_image_list):
    shutil.copy(image_path, 'valid/images')
    
for label_path in tqdm(valid_label_list):
    shutil.copy(label_path, 'valid/labels')

number of images:  10000
number of labels:  10000


100%|██████████| 1000/1000 [00:09<00:00, 104.83it/s]


In [27]:
from glob import glob
train_images = glob("/content/drive/MyDrive/Projects/BDD/train/images/*.jpg")
len(train_images)

9000

In [28]:
train_labels = glob('/content/drive/MyDrive/Projects/BDD/train/labels/*.txt')
len(train_labels)

9000

In [29]:
valid_imgs = glob('/content/drive/MyDrive/Projects/BDD/valid/images/*.jpg')
len(valid_imgs)

1000

In [30]:
valid_labels = glob('/content/drive/MyDrive/Projects/BDD/valid/labels/*.txt')
len(valid_labels)

1000

# Preparing the dataset for training

In [31]:
!pwd

/content/drive/MyDrive/Projects/BDD


In [32]:
yaml_text = """train: /content/drive/MyDrive/Projects/BDD/train/images
val: /content/drive/MyDrive/Projects/BDD/valid

nc: 1
names: ['Car']"""

In [33]:
with open("data.yaml", 'w') as f:
    f.write(yaml_text)
%cat data.yaml

train: /content/drive/MyDrive/Projects/BDD/train/images
val: /content/drive/MyDrive/Projects/BDD/valid

nc: 1
names: ['Car']

In [34]:
# check the assigned GPU type
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
#YOLO 
# !git clone https://github.com/ultralytics/yolov5  && cd yolov5 && pip install -r requirements.txt


In [ ]:
!python ./yolov5/train.py --img 512 --batch 16 --epochs 100 --workers 2 --data data.yaml --cfg "./yolov5/models/yolov5s.yaml" --name yolov5x_car --cache


train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=yolov5x_car, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-237-g55e9516 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t